<a href="https://colab.research.google.com/github/sawandarekar/era_v3_assignment_6/blob/main/ERA_V3_Session_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [113]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()
      # Attempt-2
      self.conv1 = nn.Sequential(
          nn.Conv2d(1, 32, 5, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(32),
          nn.Dropout(0.1),
          nn.Conv2d(32, 24, 3, stride=1, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(24),
          nn.Dropout(0.1)
      )

      self.conv2 = nn.Sequential(
          nn.Conv2d(24, 16, 3, stride=2, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(16),
          #nn.MaxPool2d(2, 2),
          nn.Dropout(0.1)
      )

      self.conv3 = nn.Sequential(
          nn.Conv2d(16, 24, 3, stride=2, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(24),
          nn.Dropout(0.1)
      )

      self.conv4 = nn.Sequential(
            nn.Conv2d(24,16,3,stride=1,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            # nn.MaxPool2d(2, 2),
            # nn.Dropout(0.1)
        )


      self.conv5 = nn.Sequential(
          nn.Conv2d(16, 10, 3, stride=1)
      )

      self.gap = nn.AvgPool2d(5)

      # self.fc = nn.Sequential(
      #       nn.Linear(10, 10)
      #   )


      # Attempt - 1
      # super(Net, self).__init__()
      # self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
      # self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
      # self.pool1 = nn.MaxPool2d(2, 2)
      # self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
      # self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
      # self.pool2 = nn.MaxPool2d(2, 2)
      # self.conv5 = nn.Conv2d(256, 512, 3)
      # self.conv6 = nn.Conv2d(512, 1024, 3)
      # self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)

        #x = x.view(x.size(0), -1)
        x = self.gap(x)
        # x = self.fc(x)
        x = x.view(-1, 10)
        x = F.log_softmax(x, dim=1)
        return x

        # x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        # x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        # x = F.relu(self.conv6(F.relu(self.conv5(x))))
        # x = F.relu(self.conv7(x))
        # x = x.view(-1, 10)
        # return F.log_softmax(x)

In [114]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
model


Net(
  (conv1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.1, inplace=False)
    (4): Conv2d(32, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.1, inplace=False)
  )
  (conv2): Sequential(
    (0): Conv2d(24, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.1, inplace=False)
  )
  (conv3): Sequential(
    (0): Conv2d(16, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.1, inplace=False)
  )
  (conv4): Sequential(
    (0): Conv2d(

In [115]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [116]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        #print(f'epoch:{epoch} Output shape: {output.shape}, Target shape: {target.shape}')
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'epoch={epoch} loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [117]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f"device: {device}")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             832
              ReLU-2           [-1, 32, 26, 26]               0
       BatchNorm2d-3           [-1, 32, 26, 26]              64
           Dropout-4           [-1, 32, 26, 26]               0
            Conv2d-5           [-1, 24, 26, 26]           6,936
              ReLU-6           [-1, 24, 26, 26]               0
       BatchNorm2d-7           [-1, 24, 26, 26]              48
           Dropout-8           [-1, 24, 26, 26]               0
            Conv2d-9           [-1, 16, 13, 13]           3,472
             ReLU-10           [-1, 16, 13, 13]               0
      BatchNorm2d-11           [-1, 16, 13, 13]              32
          Dropout-12           [-1, 16, 13, 13]               0
           Conv2d-13             [-1, 24, 7, 7]           3,480
             ReLU-14      

In [118]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

epoch=1 loss=0.06565283238887787 batch_id=468: 100%|██████████| 469/469 [00:26<00:00, 17.81it/s]



Test set: Average loss: 0.0863, Accuracy: 9756/10000 (97.56%)



epoch=2 loss=0.12914450466632843 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.76it/s]



Test set: Average loss: 0.0423, Accuracy: 9881/10000 (98.81%)



epoch=3 loss=0.09096860885620117 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.95it/s]


...


Test set: Average loss: 0.0176, Accuracy: 9944/10000 (99.44%)



epoch=19 loss=0.010610316880047321 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.33it/s]



Test set: Average loss: 0.0183, Accuracy: 9945/10000 (99.45%)



epoch=20 loss=0.010763414204120636 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.72it/s]



Test set: Average loss: 0.0191, Accuracy: 9949/10000 (99.49%)

